In [708]:
import decimal

import app.services.algorithm.dev
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import os

from win32.Demos.RegCreateKeyTransacted import trans

os.chdir("..\\..\\..")
print(os.getcwd())
import pulp
from app.utils.config import db_config
from app.models.player import Player
from sqlalchemy import Select
from sqlalchemy.orm import Session
from app.models.squad import Squad
from app.services.algorithm.utils import encode_squad



C:\


In [709]:
engine=create_engine(db_config.DB_CONFIG)
with Session(engine) as session:
    with session.begin():
        data=session.execute(Select(Player).order_by(Player.id))
        data=data.scalars().all()
        playersNum=len(data)
        df=pd.DataFrame.from_records([d.to_dict() for d in data])
        squad=session.execute(Select(Squad).where(Squad.id==9104)).scalars().first()
        current_squad=encode_squad(playersNum,squad)


In [710]:
cur_squad_arr=(current_squad==1).nonzero()

In [715]:
df.head

<bound method NDFrame.head of       id                          name  team_id  position  price  \
0      1         Fábio Ferreira Vieira        1         3     54   
1      2     Gabriel Fernando de Jesus        1         4     68   
2      3  Gabriel dos Santos Magalhães        1         2     61   
3      4                   Kai Havertz        1         4     80   
4      5                     Karl Hein        1         1     40   
..   ...                           ...      ...       ...    ...   
673  674                 Paris Maghoma        4         3     45   
674  675           Josh Wilson-Esbrand       13         2     40   
675  676                    Zach Marsh        7         4     45   
676  677                 Ollie Scarles       19         2     40   
677  678                  Callum Bates        8         3     45   

    expectedPoints  availability  
0     [0.00, 0.00]             0  
1     [1.20, 1.70]           100  
2     [2.00, 2.50]           100  
3     [1.50, 

In [712]:
def add_transfer_constrains(df,model,free_in, paid_in, transfer_out, transfer_in, current_squad, budget_now,freeTransfers):
        model += sum(free_in) <= freeTransfers
        in_cost = sum(transfer_in * df.price.tolist())
        out_cost = sum(transfer_out * df.price.tolist())
        budget_next_week = budget_now + out_cost - in_cost
        model += budget_next_week >= 0

In [713]:
def add_positions_and_pl_teams_constrains(df,model,squad,team,subs,captain):
    positions=df.position.tolist()
    teams_ids=df.team_id.tolist()
    for i in range(0,playersNum):
        model+=squad[i]-subs[i]>=0
        model+=team[i]-captain[i]>=0
        model+=team[i]+subs[i]<=1
        
    model+=sum(squad)==15
    model+= sum(team)==11
    model+=sum(captain)==1
    model+=sum(subs)==4
    #goalkeeper
    model=+ sum(squadmate for squadmate,position in zip(squad,positions) if position==1 )==2
    model+= sum(teammate for teammate,position in zip(team,positions) if position==1 )==1
    #defender
    
    model=+ sum(squadmate for squadmate,position in zip(squad,positions) if position==2 )==5
    model+= sum(teammate for teammate,position in zip(team,positions) if position==2 )>=3
    model+=sum(teammate for teammate,position in zip(team,positions) if position==2 )<=5
    #midfielder
    
    model=+ sum(squadmate for squadmate,position in zip(squad,positions) if position==3 )==5
    model+= sum(teammate for teammate,position in zip(team,positions) if position==3 )>=3
    model+=sum(teammate for teammate,position in zip(team,positions) if position==3 )<=5
    #forward
    model+= sum(teammate for teammate,position in zip(team,positions) if position==4 )>=1
    model+=sum(teammate for teammate,position in zip(team,positions) if position==4 )<=3
    model=+ sum(squadmate for squadmate,position in zip(squad,positions) if position==4 )==3
    #clubs
    pl_teams=df.team_id.unique()
    for pl_team in pl_teams:
        model+=sum(squadmate for squadmate,team_id in zip(squad,teams_ids) if team_id==pl_team)<=3

In [714]:
def add_objective(model,captain,team,squad,subs,paid_in,expected_points):
    penalty= sum(paid_in)*4
    obj=sum(team*expected_points)+sum(captain*expected_points)-penalty
    model+=obj,"Objective"

In [668]:
def solve(model,df,budget_now,current_squad,freeTransfers,playersNum):
    free_in, paid_in, transfer_out, transfer_in, captain,subs=get_decision_arrays(playersNum)
    squad=current_squad+transfer_in-transfer_out
    team=squad-subs
    add_transfer_constrains(df,model,free_in, paid_in, transfer_out, transfer_in, current_squad, budget_now,freeTransfers)
    
    expected_points=np.array(pd.DataFrame(df.expectedPoints.values.tolist(), index=df.id).apply(pd.to_numeric,downcast="float").iloc[:,0].tolist())
    add_positions_and_pl_teams_constrains(df,model,squad,team,subs,captain)
    add_objective(model,captain,team,squad,subs,paid_in,expected_points)
    model.solve()
    return transfer_in,transfer_out,captain,subs,team,squad

In [703]:
model=pulp.LpProblem("FPL", pulp.LpMaximize)

In [704]:
transfer_in,transfer_out,captain,subs,team,squad=solve(model,df,21,current_squad,1,playersNum)

In [705]:
print("squad")
tmps=see_result(squad,playersNum)
print("subs")
tmp=see_result(subs,playersNum)
print("team")
tmpt=see_result(team,playersNum)
print("in")
tmp1=see_result(transfer_in,playersNum)
print("out")
tmp2=see_result(transfer_out,playersNum)
print("captain")
tmp3=see_result(captain,playersNum)

squad
98 1.0, 116 1.0, 254 1.0, 290 1.0, 308 1.0, 325 1.0, 327 1.0, 332 1.0, 349 1.0, 350 1.0, 365 1.0, 412 1.0, 446 1.0, 520 1.0, 540 1.0, 
subs
116 1.0, 290 1.0, 308 1.0, 332 1.0, 
team
98 1.0, 254 1.0, 325 1.0, 327 1.0, 349 1.0, 350 1.0, 365 1.0, 412 1.0, 446 1.0, 520 1.0, 540 1.0, 
in
349 1.0, 365 1.0, 540 1.0, 
out
2 1.0, 179 1.0, 229 1.0, 
captain
327 1.0, 


In [707]:
df.iloc[tmps].position.value_counts()

position
3    5
2    5
4    3
1    2
Name: count, dtype: int64

In [662]:
for elem in tmpt:
    if elem not in tmps:
        print("FUck")

FUck
FUck
FUck
FUck
FUck
FUck
FUck
FUck


In [367]:
def see_result(arr,numPlayers):
    res=[]
    for i in range(0,numPlayers):
        if arr[i].value()!=0:
            res.append(i)
            print(i,arr[i].value(), end=", ")
    print()
    return res        

In [585]:
for s in subs:
    print(s.value())

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [681]:
df.iloc[tmp1].price.sum()

np.int64(215)

In [682]:
df.iloc[tmp2].price.sum()

np.int64(197)

In [550]:
df.iloc[tmp2].price.sum()

np.int64(197)

In [250]:
cur_squad_arr

(array([  2,  98, 116, 179, 229, 254, 290, 308, 325, 327, 332, 350, 412,
        446, 520]),)

In [117]:
pd.DataFrame(df.expectedPoints.values.tolist(),index=df.id).apply(pd.to_numeric,downcast="float")

,0,1
id,,
1,0.0,0.0
2,1.2,1.7
3,2.0,2.5
4,1.5,2.5
5,0.0,0.0
...,...,...
674,0.0,0.0
675,2.2,1.7
676,1.0,1.0


In [99]:
for tm,pos in zip(current_squad,df.position):
    if tm==1:
        print(pos)

2
3
3
4
3
2
2
3
2
3
2
4
1
4
1


In [97]:
df.position.tolist()

[3,
 4,
 2,
 4,
 1,
 2,
 3,
 2,
 3,
 3,
 4,
 3,
 3,
 1,
 1,
 3,
 3,
 2,
 3,
 3,
 2,
 2,
 3,
 2,
 2,
 2,
 4,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 2,
 2,
 3,
 4,
 1,
 2,
 3,
 3,
 2,
 2,
 2,
 1,
 1,
 3,
 2,
 2,
 1,
 2,
 3,
 3,
 1,
 2,
 3,
 4,
 2,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 2,
 4,
 2,
 3,
 2,
 1,
 3,
 1,
 3,
 3,
 3,
 2,
 3,
 2,
 4,
 3,
 1,
 2,
 2,
 3,
 2,
 3,
 3,
 1,
 2,
 2,
 3,
 3,
 2,
 3,
 3,
 3,
 2,
 3,
 3,
 3,
 2,
 2,
 3,
 4,
 4,
 1,
 4,
 3,
 2,
 3,
 3,
 3,
 2,
 3,
 3,
 3,
 2,
 3,
 2,
 4,
 3,
 3,
 3,
 3,
 2,
 4,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 2,
 3,
 3,
 1,
 4,
 2,
 2,
 1,
 2,
 4,
 3,
 3,
 3,
 1,
 2,
 1,
 1,
 4,
 3,
 3,
 2,
 2,
 3,
 2,
 2,
 4,
 4,
 3,
 2,
 3,
 3,
 2,
 2,
 2,
 3,
 3,
 4,
 2,
 3,
 4,
 3,
 4,
 3,
 3,
 1,
 3,
 1,
 3,
 2,
 3,
 2,
 3,
 2,
 3,
 3,
 2,
 2,
 2,
 3,
 4,
 3,
 2,
 1,
 2,
 3,
 1,
 3,
 3,
 4,
 3,
 1,
 2,
 2,
 3,
 3,
 3,
 2,
 3,
 3,
 4,
 2,
 4,
 2,
 3,
 3,
 3,
 2,
 3,
 1,
 2,
 4,
 3,
 2,
 4,
 3,
 2,
 1,
 2,
 4,
 2,
 3,
 3,
 2,
 1,
 3,
 2,
 2,
 3,
 3,
 1,
 3,
 2,


In [689]:
cur_squad_arr

(array([  2,  98, 116, 179, 229, 254, 290, 308, 325, 327, 332, 350, 412,
        446, 520]),)

In [690]:
transfer_in_decisions, transfer_out_decisions, starters, sub_decisions, captain_decisions = opt.solve(cur_squad_arr, budget_now=0, sub_factor=0)

C:\Users\18pep\Documents\sem7\inżynierka\BackEnd\.venv\Lib\site-packages\pulp\pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solver status: 1


In [683]:
import importlib
importlib.reload(app.services.algorithm.solver)
from app.services.algorithm.solver import Solver


<module 'app.services.algorithm.solver' from 'C:\\Users\\18pep\\Documents\\sem7\\inżynierka\\BackEnd\\app\\services\\algorithm\\solver.py'>

In [641]:
solver=Solver(df,current_squad,playersNum,21,3)

In [642]:
transfer_in,transfer_out,captain,subs,team,squad=solver.solve()

In [643]:
print("squad")
tmps=see_result(squad,playersNum)
print("subs")
tmp=see_result(subs,playersNum)
print("team")
tmpt=see_result(team,playersNum)
print("in")
tmp1=see_result(transfer_in,playersNum)
print("out")
tmp2=see_result(transfer_out,playersNum)
print("captain")
tmp3=see_result(captain,playersNum)

squad
0 -1.0, 1 -1.0, 3 -1.0, 4 -1.0, 5 -1.0, 6 -1.0, 7 -1.0, 8 -1.0, 9 -1.0, 10 -1.0, 11 -1.0, 12 -1.0, 13 -1.0, 14 -1.0, 15 -1.0, 16 -1.0, 17 -1.0, 18 -1.0, 19 -1.0, 20 -1.0, 21 -1.0, 22 -1.0, 23 -1.0, 24 -1.0, 25 -1.0, 26 -1.0, 27 -1.0, 28 -1.0, 29 -1.0, 30 -1.0, 31 -1.0, 32 -1.0, 33 -1.0, 34 -1.0, 35 -1.0, 36 -1.0, 37 -1.0, 38 -1.0, 39 -1.0, 40 -1.0, 41 -1.0, 42 -1.0, 43 -1.0, 44 -1.0, 45 -1.0, 46 -1.0, 47 -1.0, 48 -1.0, 49 -1.0, 50 -1.0, 51 -1.0, 52 -1.0, 53 -1.0, 54 -1.0, 55 -1.0, 56 -1.0, 57 -1.0, 58 -1.0, 59 -1.0, 60 -1.0, 61 -1.0, 62 -1.0, 63 -1.0, 64 -1.0, 65 -1.0, 66 -1.0, 67 -1.0, 68 -1.0, 69 -1.0, 70 -1.0, 71 -1.0, 72 -1.0, 73 -1.0, 74 -1.0, 75 -1.0, 76 -1.0, 77 -1.0, 78 -1.0, 79 -1.0, 80 -1.0, 81 -1.0, 82 -1.0, 83 -1.0, 84 -1.0, 85 -1.0, 86 -1.0, 87 -1.0, 88 -1.0, 89 -1.0, 90 -1.0, 91 -1.0, 92 -1.0, 93 -1.0, 94 -1.0, 95 -1.0, 96 -1.0, 97 -1.0, 99 -1.0, 100 -1.0, 101 -1.0, 102 -1.0, 103 -1.0, 104 -1.0, 105 -1.0, 106 -1.0, 107 -1.0, 108 -1.0, 109 -1.0, 110 -1.0, 111 -1.0, 1